In [3]:
const _ = require('lodash');

## chunk

Splits a list of values into quantiles. Equivalent to `np.array_split` in Python. Useful, the native equivalent is a curmudgeon.

In [46]:
_.chunk(['a', 'b', 'c'], size=1)

[ [ 'a' ], [ 'b' ], [ 'c' ] ]

In [6]:
_.chunk(['a', 'b', 'c'], size=2)

[ [ 'a', 'b' ], [ 'c' ] ]

In [73]:
_.chunk(['a', 'b', 'c'], size=3)

[ [ 'a', 'b', 'c' ] ]

In [67]:
function chunk(vals, size=1) {
    if (size >= vals.length) {
        return [vals];
    } else {
        return (
            [...Array(Math.round(vals.length / size)).keys()]
                .map(i => [i * size, (i + 1 * size)]))  // generate index pairs
                .map(idxs => vals.slice(idxs[0], idxs[1]))
    }
}

In [69]:
chunk(['a', 'b', 'c'], size=1)

[ [ 'a' ], [ 'b' ], [ 'c' ] ]

In [70]:
chunk(['a', 'b', 'c'], size=2)

[ [ 'a', 'b' ], [ 'c' ] ]

In [71]:
chunk(['a', 'b', 'c'], size=3)

[ [ 'a', 'b', 'c' ] ]

## compact

Removes falsy elements from a list. You're better off doing this natively, with `filter`.

In [5]:
_.compact(['a', 'b', null])

[ 'a', 'b' ]

In [76]:
function compact(vals) {
    return vals.filter(v => v)
}

In [77]:
compact(['a', 'b', null])

[ 'a', 'b' ]

## concat

Contatenates lists of various sizes. The same as the native method, just pulled out to the top level, don't recommend.

In [80]:
_.concat('a', ['b', 'c'], ['d'])

[ 'a', 'b', 'c', 'd' ]

In [91]:
[].concat('a', ['b', 'c'], ['d'])

[ 'a', 'b', 'c', 'd' ]

## difference

Set difference between the first array input and all of the rest of the array input. Useful.

In [183]:
_.difference([1, 1, 2], [2, 3, 4])

[ 1, 1 ]

In [190]:
function difference(target, ...other) {
    const s = new Set(target);
    [].concat(...other).forEach(v => s.delete(v));
    return target.filter(t => s.has(t))
}

In [193]:
difference([1, 2], [2, 3], [3, 4])

[ 1 ]

In [194]:
difference([1, 1, 2], [2, 3], [3, 4])

[ 1, 1 ]

## differenceBy

Set difference with a pre-comparator transform. Recommend using `differenceWith` instead.

In [125]:
_.differenceBy([1.2, 2.1], [2.3, 3.1], Math.floor)

[ 1.2 ]

## differenceWith

Set difference with a custom comparator. Useful.

In [163]:
_._.differenceWith([1, 2], [2, 3], 
                   (a, b) => a === b)

[ 1 ]

In [197]:
function differenceWith(target, ...args) {
    const resultset = new Set(target);
    const other = new Set([].concat(...args.slice(0, args.length - 1)));
    const comparator = args[args.length - 1];
    
    // Can be made better than O(n^2) by sorting and tracking indicial values.
    // But this is easy-to-write.
    resultset.forEach(t => {
        other.forEach(o => {
            comparator(t, o) ? resultset.delete(t) : null;
        })
    })
    
    return target.filter(t => resultset.has(t))
}

In [198]:
differenceWith([1, 2], [2, 3], (a, b) => a === b)

[ 1 ]

In [199]:
differenceWith([1, 1, 2], [2, 3], (a, b) => a === b)

[ 1, 1 ]

## fill

Fills values between a start and end index. Useless, copies an existing array method.

In [202]:
_.fill([1, 2, 3, 4, 5], "*", start=1, end=4)

[ 1, '*', '*', '*', 5 ]

In [259]:
function fill(arr, value, start=0, end=arr.length) {
    return arr.map((prior, i) => (start <= i) & (i < end) ? value : prior);
}

In [262]:
[1, 2, 3, 4, 5].fill("*", 1, 4)

[ 1, '*', '*', '*', 5 ]

## find, findIndex, and findLastIndex

Also copy existing array methods. The last one is just `reverse`, *then* `findIndex`.

In [265]:
[1, 2, {foo: 2}, 4].find(v => v.foo === 2)

{ foo: 2 }

In [266]:
[1, 2, {foo: 2}, 4].findIndex(v => v.foo === 2)

2

In [270]:
[{foo: 2}, {foo: 2}, 3, 4].reverse().findIndex(v => v.foo === 2)

2

## flatten, flattenDeep, flattenDepth

The latter is worse than the native `concat`, whilst the former two are useful. The last one has a limit on how far it will unfurl the array.

In [271]:
_.concat([1], [2], [3])

[ 1, 2, 3 ]

In [273]:
_.flatten([[1], [2], [3]])

[ 1, 2, 3 ]

In [276]:
_.flattenDeep([[1], [[[[[[2]]]]]], [3]])

[ 1, 2, 3 ]

In [283]:
_.flattenDepth([[1], [[[[[[2]]]]]], [3]], 4)

[ 1, [ [ 2 ] ], 3 ]

## fromPairs

Converts between key-value pair array and object. Useful, but this is such a rarely used, easy-to-write function that I doubt I'll remember to use this func.

In [284]:
_.fromPairs([['a', 1], ['b', 2]]);

{ a: 1, b: 2 }

In [336]:
function fromPairs(pairs) {
    let result = {}
    pairs.forEach(pair => result[pair[0]] = pair[1]);
    return result;
}

In [337]:
fromPairs([['a', 1], ['b', 2]]);

{ a: 1, b: 2 }

## intersection and intersectionWith

Generates a list of unique values included in all arrays. Amazing this isn't a JS built-in, but it's not. Extremely useful, obviously.

In [339]:
_.intersection([2, 1, 3], [2, 3])

[ 2, 3 ]

In [403]:
function intersection(...arrs) {
    function pairwise_intersection(a, b) { return new Set([...a].filter(v => b.has(v))) }
    
    let result = new Set(arrs[0]);
    for (b_idx of _.range(1, arrs.length)) {
        let b = new Set(arrs[b_idx]);
        result = pairwise_intersection(result, b);
    }
    
    return result;
}

In [407]:
var objects = [{ 'x': 1, 'y': 2 }, { 'x': 2, 'y': 1 }];
var others = [{ 'x': 1, 'y': 1 }, { 'x': 1, 'y': 2 }];
 
_.intersectionWith(objects, others, _.isEqual);

[ { x: 1, y: 2 } ]

## zip

Not a built-in, unlike Python, and as always pretty freaking useful, so a must have from lodash.

In [430]:
function zip(...arrs) {
    const len = Math.max(...arrs.map(arr => arr.length));
    let result = [];
    for (idx of [...Array(len).keys()]) {
        result.push(arrs.map((arr) => arr[idx]));
    }
    return result;
}

In [431]:
zip([1, 2], [3, 4])

[ [ 1, 3 ], [ 2, 4 ] ]

## countby

Basically `pd.DataFrame.value_counts`! Super useful.

In [434]:
_.countBy([6, 4, 6], Math.floor);

{ '4': 1, '6': 2 }

In [433]:
_.countBy([6.1, 4.2, 6.3], Math.floor);

{ '4': 1, '6': 2 }

In [468]:
function countBy(arr, interatee=v => v) {
    let result = {};
    let existing = new Set();
    arr.forEach(v => {
        if (existing.has(v)) {
            result[v] += 1;
        } else {
            existing.add(v);
            result[v] = 1;
        }
    });
    return result
}

In [470]:
countBy([6, 4, 6, 6], Math.floor)

{ '4': 1, '6': 3 }

## find

Just a copy of the `find` built-in.

In [4]:
_.find([{'a': 1, 'b': 2}, {'a': 2, 'b': 3}, {'a': 2, 'b': 4}], obj => obj.a === 2)

{ a: 2, b: 3 }

In [5]:
function find(values, predicate=v => v) {
    let i = 0;
    while (i < values.length) {
        if (predicate(values[i])) { return values[i] }
        i += 1;
    }
}

In [6]:
find([{'a': 1, 'b': 2}, {'a': 2, 'b': 3}, {'a': 2, 'b': 4}], obj => obj.a === 2)

{ a: 2, b: 3 }

In [7]:
[{'a': 1, 'b': 2}, {'a': 2, 'b': 3}, {'a': 2, 'b': 4}].find(obj => obj.a === 2)

{ a: 2, b: 3 }

## groupBy

Just like a `pandas` `groupby`!

In [8]:
_.groupBy(['one', 'two', 'three'], v => v.length);

{ '3': [ 'one', 'two' ], '5': [ 'three' ] }

## keyBy

Transforms an array into a keyed map, using some key-defining function. Sometimes useful.

In [11]:
var array = [
  { 'dir': 'left', 'code': 97 },
  { 'dir': 'right', 'code': 100 }
];

_.keyBy(array, obj => obj.dir);

{ left: { dir: 'left', code: 97 },
  right: { dir: 'right', code: 100 } }

In [29]:
function keyBy(arr, iteratee=v => v) {
    let result = {};
    arr.forEach(v => { 
        let k = iteratee(v); 
        result[k] = v;
    });
    return result;
}

In [30]:
var array = [
  { 'dir': 'left', 'code': 97 },
  { 'dir': 'right', 'code': 100 }
];
 
keyBy(array, obj => obj.dir);

{ left: { dir: 'left', code: 97 },
  right: { dir: 'right', code: 100 } }

## partition

Partitions a list into two parts using some predicate. Sometimes useful, but easy to implement on your own.

In [31]:
var users = [
  { 'user': 'barney',  'age': 36, 'active': false },
  { 'user': 'fred',    'age': 40, 'active': true },
  { 'user': 'pebbles', 'age': 1,  'active': false }
];
 
_.partition(users, function(o) { return o.active; });

[ [ { user: 'fred', age: 40, active: true } ],
  [ { user: 'barney', age: 36, active: false },
    { user: 'pebbles', age: 1, active: false } ] ]

In [34]:
function partition(arr, predicate=v => v) {
    let result = [[], []];
    arr.forEach(v => predicate(v) ? result[0].push(v) : result[1].push(v));
    return result
}

In [35]:
partition(users, function(o) { return o.active; });

[ [ { user: 'fred', age: 40, active: true } ],
  [ { user: 'barney', age: 36, active: false },
    { user: 'pebbles', age: 1, active: false } ] ]